# Facial Emotion Recognition using CNNs on FER-2013 Dataset

# Author: Abd Ur Rehman
# Date: 12_01_2024


# Project Purpose

## Objective:
### Develop a Facial Emotion Recognition model using Convolutional Neural Networks (CNNs) to classify facial emotions
### into one of seven categories (Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral) using the FER-2013 dataset.

## Dataset:
### The FER-2013 dataset consists of 35,887 grayscale images (48x48 pixels) categorized into 7 different emotions.
### This dataset provides a rich source of labeled facial expressions for training the CNN model.

## Key Tasks:
### 1. Data Preprocessing: Load, clean, and preprocess the images, one-hot encode the labels.
### 2. Model Building: Construct a CNN-based neural network for image classification.
### 3. Model Training: Train the model on the FER-2013 dataset to learn patterns in facial expressions.
### 4. Evaluation: Assess the model's accuracy and performance using test data, visualize accuracy and loss graphs.
### 5. Application: Implement a prediction function to classify emotions from new images.

## Expected Outcome:
### By the end of this project, we aim to have a functional Facial Emotion Recognition model capable of classifying 
### facial expressions into one of seven emotions. This model could serve as a foundation for applications such as 
### emotion detection in video surveillance, human-computer interaction, or emotional AI.


## Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns


# Load the FER-2013 dataset

In [ ]:

data = pd.read_csv('fer2013.csv')


## Data Preprocessing

In [ ]:
# Reshape the image data and normalize pixel values
x = np.array([np.fromstring(image, sep=' ').reshape(48, 48, 1) for image in data['pixels']])
y = to_categorical(data['emotion'], num_classes=7)

In [ ]:
# Normalize the pixel values to range [0, 1]
x = x / 255.0

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
# Print the shape of the data
print(f'Training data shape: {x_train.shape}')
print(f'Testing data shape: {x_test.shape}')

## Build the CNN model

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout to prevent overfitting
    layers.Dense(7, activation='softmax')  # 7 classes for 7 emotions
])

## Compile the model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Print model summary

In [ ]:

model.summary()

## Train the model

In [ ]:

epochs = 50
batch_size = 64
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))



## Save the trained model

In [ ]:

model.save('emotion_recognition_model.h5')

## Evaluate the model

In [ ]:

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')

## Plot the accuracy and loss graphs

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.title('Training and Validation Loss')

plt.show()

## Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)

## Print the classification report

In [ ]:

print(classification_report(y_true, y_pred, target_names=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']))


## Plot confusion matrix

In [ ]:

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'], yticklabels=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

## Prediction Function

In [ ]:

def predict_emotion(model, img):
    """
    Predicts the emotion of a given image.

    Parameters:
    model -- trained CNN model
    img -- image to be predicted, in the shape (48, 48, 1)

    Returns:
    The predicted emotion as a string
    """
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = img / 255.0  # Normalize
    
    # Predict emotion
    predictions = model.predict(img)
    emotion_index = np.argmax(predictions)
    
    # Emotion labels
    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    return emotions[emotion_index]

## Example usage of the prediction function

In [ ]:

sample_img = x_test[0]  # Choose a sample image from the test set
predicted_emotion = predict_emotion(model, sample_img)
print(f'Predicted emotion: {predicted_emotion}')